In [67]:
import json
import re
from bert_score import score
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

In [265]:
associations_valid = [('data/test_data/few_shot/test_data_2024_abstract_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2024_abstract_prompts_fewshot_gpt4o_results.jsonl', 'GPT4o-Abstract-Fewshot'),
                      ('data/test_data/few_shot/test_data_2024_abstract_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2024_abstract_prompts_oneshot_gpt4o_results.jsonl', 'GPT4o-Abstract-Oneshot'),
                      ('data/test_data/few_shot/test_data_2024_abstract_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2024_abstract_prompts_gpt4o_results.jsonl', 'GPT4o-Abstract-Zeroshot'),
                      ('data/test_data/few_shot/test_data_2024_summary_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2024_summary_prompts_fewshot_gpt4o_results.jsonl', 'GPT4o-Summary-Fewshot'),
                      ('data/test_data/few_shot/test_data_2024_summary_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2024_summary_prompts_oneshot_gpt4o_results.jsonl', 'GPT4o-Summary-Oneshot'),
                      ('data/test_data/few_shot/test_data_2024_summary_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2024_summary_prompts_gpt4o_results.jsonl', 'GPT4o-Summary-Zeroshot')]

associations_test = [('data/test_data/few_shot/test_data_2025_abstract_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2025_abstract_prompts_fewshot_gpt4o_results.jsonl', 'GPT4o-Abstract-Fewshot'),
                     ('data/test_data/few_shot/test_data_2025_abstract_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2025_abstract_prompts_oneshot_gpt4o_results.jsonl', 'GPT4o-Abstract-Oneshot'),
                     ('data/test_data/few_shot/test_data_2025_abstract_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2025_abstract_prompts_gpt4o_results.jsonl', 'GPT4o-Abstract-Zeroshot'),
                     ('data/test_data/few_shot/test_data_2025_summary_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2025_summary_prompts_fewshot_gpt4o_results.jsonl', 'GPT4o-Summary-Fewshot'),
                     ('data/test_data/few_shot/test_data_2025_summary_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2025_summary_prompts_oneshot_gpt4o_results.jsonl', 'GPT4o-Summary-Oneshot'),
                     ('data/test_data/few_shot/test_data_2025_summary_prompts_fewshot.jsonl', 'results/gpt4o/test_data_answer_not_in_prompt/test_data_2025_summary_prompts_gpt4o_results.jsonl', 'GPT4o-Summary-Zeroshot')]

In [266]:
def get_gold_and_predictions(association):
    with open(association[0], 'r') as f:
        gold_standard = [json.loads(line) for line in f]
        #gold_standard = json.load(f)

    with open(association[1], 'r') as f:
        predictions = [json.loads(line) for line in f]

    gold_standard = [{item['paper_id']:json.loads(item['messages'][-1]['content'])} for item in gold_standard]
    predictions = [{item['paper_id']:item['review']} for item in predictions if 'review' in item.keys()]
    
    return gold_standard, predictions

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [328]:
gold_standard, predictions = get_gold_and_predictions(associations_valid[0])
prediction = list(predictions[0].values())[0]
#list([item for item in gold_standard if item.keys() == predictions[0].keys()][0].values())[0]
arr = np.array([])
arr = np.append(arr, [prediction[cat] for cat in numerical_scores])
np.append(arr, [prediction[cat] for cat in numerical_scores]).reshape(-1,5)

array([[3., 3., 3., 7., 4.],
       [3., 3., 3., 7., 4.]])

In [314]:
mean_squared_error([1,7, 0],[2,3, 0])

5.666666666666667

In [354]:
gold_text[:,0].tolist()

['The paper proposes a scheme to adapt gradient-based methods for weight-entangled spaces in neural architecture search (NAS). This integration of weight-entanglement and gradient-based NAS is a new approach that has not been explored before. The paper also presents a comprehensive evaluation of the properties of single and two-stage approaches, including any-time performance, memory consumption, robustness to training fraction, and the effect of fine-tuning.',
 '1. Contrastive learning has demonstrated promise in the field of computer vision; this paper shows a novel application of this concept to the field of Biology.\n\n2. The paper has a strong theoretical foundation and is well presented. \n\n3. Quantitative results are convincing and promising.',
 '1. This paper is overall clearly clarified and well organized.\n2. The use of adapters for lifelong test-time adaptation seems novel.',
 'The authors introduce a method that selects features based on neural estimation of mutual informa

In [365]:
str(torch.mean(score(gold_text[:,0].tolist(), pred_text[:,0].tolist(), lang="en")[0]))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'tensor(0.8499)'

In [366]:
def calc_bertscore(gold_cat, pred_cat):
    bert = score(gold_cat.tolist(), pred_cat.tolist(), lang="en")
    mean_precision = torch.mean(bert[0])
    mean_recall = torch.mean(bert[1])
    mean_f1 = torch.mean(bert[2])
    return mean_precision, mean_recall, mean_f1

In [368]:
numerical_scores = ["Soundness", "Presentation", "Contribution", "Rating", "Confidence"]
text_fields = ["Strengths", "Weaknesses", "Questions"]
for association in associations_test:
    gold_standard, predictions = get_gold_and_predictions(association)

    gold_scores = []
    pred_scores = []

    gold_text = []
    pred_text = []
    for prediction in predictions:
        gold = list([item for item in gold_standard if item.keys() == prediction.keys()][0].values())[0]
        prediction = list(predictions[0].values())[0]
        gold_scores.append([gold[category] for category in numerical_scores])
        pred_scores.append([prediction[category] for category in numerical_scores])

        gold_text.append([gold[category] for category in text_fields])
        pred_text.append([prediction[category] for category in text_fields])
    gold_scores = np.array(gold_scores)
    pred_scores = np.array(pred_scores)

    gold_text = np.array(gold_text)
    pred_text = np.array(pred_text)

    mean_precision_strengths, mean_recall_strengths, mean_f1_strengths = calc_bertscore(gold_text[:,0], pred_text[:,0])
    mean_precision_weaknesses, mean_recall_weaknesses, mean_f1_weaknesses = calc_bertscore(gold_text[:,1], pred_text[:,1])
    mean_precision_questions, mean_recall_questions, mean_f1_questions = calc_bertscore(gold_text[:,2], pred_text[:,2])

    print(association[2] + ' MSE_soundness = ' + str(mean_squared_error(gold_scores[:,0], pred_scores[:,0])))
    print(association[2] + ' MSE_presentation = ' + str(mean_squared_error(gold_scores[:,1], pred_scores[:,1])))
    print(association[2] + ' MSE_contribution = ' + str(mean_squared_error(gold_scores[:,2], pred_scores[:,2])))
    print(association[2] + ' MSE_rating = ' + str(mean_squared_error(gold_scores[:,3], pred_scores[:,3])))
    print(association[2] + ' MSE_confidence = ' + str(mean_squared_error(gold_scores[:,4], pred_scores[:,4])))
    print(association[2] + ' Strengths_BertScore = ' + str(mean_precision_strengths) + ' / ' + str(mean_recall_strengths) + ' / ' + str(mean_f1_strengths))
    print(association[2] + ' Weaknesses_BertScore = ' + str(mean_precision_weaknesses) + ' / ' + str(mean_recall_weaknesses) + ' / ' + str(mean_f1_weaknesses))
    print(association[2] + ' Questions_BertScore = ' + str(mean_precision_questions) + ' / ' + str(mean_recall_questions) + ' / ' + str(mean_f1_questions))
    print('\n')


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT4o-Abstract-Fewshot MSE_soundness = 0.56
GPT4o-Abstract-Fewshot MSE_presentation = 0.63
GPT4o-Abstract-Fewshot MSE_contribution = 0.71
GPT4o-Abstract-Fewshot MSE_rating = 5.6
GPT4o-Abstract-Fewshot MSE_confidence = 0.95
GPT4o-Abstract-Fewshot Strengths_BertScore = tensor(0.8437) / tensor(0.8397) / tensor(0.8415)
GPT4o-Abstract-Fewshot Weaknesses_BertScore = tensor(0.8116) / tensor(0.8472) / tensor(0.8288)
GPT4o-Abstract-Fewshot Questions_BertScore = tensor(0.8220) / tensor(0.8298) / tensor(0.8255)




Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT4o-Abstract-Oneshot MSE_soundness = 2.1818181818181817
GPT4o-Abstract-Oneshot MSE_presentation = 0.6363636363636364
GPT4o-Abstract-Oneshot MSE_contribution = 2.757575757575758
GPT4o-Abstract-Oneshot MSE_rating = 9.929292929292929
GPT4o-Abstract-Oneshot MSE_confidence = 0.8181818181818182
GPT4o-Abstract-Oneshot Strengths_BertScore = tensor(0.8437) / tensor(0.8412) / tensor(0.8423)
GPT4o-Abstract-Oneshot Weaknesses_BertScore = tensor(0.8128) / tensor(0.8592) / tensor(0.8351)
GPT4o-Abstract-Oneshot Questions_BertScore = tensor(0.8077) / tensor(0.8265) / tensor(0.8166)




Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

GPT4o-Summary-Fewshot MSE_soundness = 2.2
GPT4o-Summary-Fewshot MSE_presentation = 0.63
GPT4o-Summary-Fewshot MSE_contribution = 0.71
GPT4o-Summary-Fewshot MSE_rating = 9.92
GPT4o-Summary-Fewshot MSE_confidence = 0.81
GPT4o-Summary-Fewshot Strengths_BertScore = tensor(0.8391) / tensor(0.8399) / tensor(0.8394)
GPT4o-Summary-Fewshot Weaknesses_BertScore = tensor(0.8060) / tensor(0.8489) / tensor(0.8267)
GPT4o-Summary-Fewshot Questions_BertScore = tensor(0.8153) / tensor(0.8239) / tensor(0.8192)




Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT4o-Summary-Oneshot MSE_soundness = 2.2222222222222223
GPT4o-Summary-Oneshot MSE_presentation = 2.2222222222222223
GPT4o-Summary-Oneshot MSE_contribution = 2.787878787878788
GPT4o-Summary-Oneshot MSE_rating = 10.02020202020202
GPT4o-Summary-Oneshot MSE_confidence = 0.8181818181818182
GPT4o-Summary-Oneshot Strengths_BertScore = tensor(0.8519) / tensor(0.8455) / tensor(0.8486)
GPT4o-Summary-Oneshot Weaknesses_BertScore = tensor(0.8153) / tensor(0.8536) / tensor(0.8338)
GPT4o-Summary-Oneshot Questions_BertScore = tensor(0.8188) / tensor(0.8281) / tensor(0.8230)




Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT4o-Summary-Zeroshot MSE_soundness = 2.2
GPT4o-Summary-Zeroshot MSE_presentation = 2.21
GPT4o-Summary-Zeroshot MSE_contribution = 2.77
GPT4o-Summary-Zeroshot MSE_rating = 9.92
GPT4o-Summary-Zeroshot MSE_confidence = 0.81
GPT4o-Summary-Zeroshot Strengths_BertScore = tensor(0.8459) / tensor(0.8564) / tensor(0.8510)
GPT4o-Summary-Zeroshot Weaknesses_BertScore = tensor(0.8072) / tensor(0.8599) / tensor(0.8325)
GPT4o-Summary-Zeroshot Questions_BertScore = tensor(0.8128) / tensor(0.8449) / tensor(0.8281)




In [135]:
def parse_review(text):
    text = text.replace('\nExplanation:\n', '',1)
    review_dict = {}
    
    # Split text into sections using double newlines as a separator
    sections = re.split(r"\n\n", text.strip())

    for section in sections:
        lines = section.split("\n")
        #lines = section
        
        # First part contains numerical ratings
        if ": " in lines[0]:
            for line in lines:
                if ": " in line:
                    key, value = line.split(": ", 1)
                    review_dict[key.strip()] = int(value) if value.isdigit() else value.strip()
        
        # Later parts contain explanations, strengths, weaknesses, questions
        else:
            key = lines[0].replace(":", "").strip()  # Remove colons and extra spaces
            review_dict[key] = "\n".join(lines[1:]).strip()  # Store the rest of the section

    return review_dict

In [136]:
import re

def parse_review1(text):
    # Define regex patterns for extracting different sections
    score_pattern = r"(\w+): (\d+)"
    section_pattern = r"(Strengths|Weaknesses|Questions):\n"

    # Initialize dictionary
    review_dict = {}

    # Extract numerical scores
    scores = re.findall(score_pattern, text)
    for key, value in scores:
        review_dict[key] = int(value)  # Convert scores to integers

    # Extract text sections
    sections = re.split(section_pattern, text)
    
    # Process sections
    for i in range(1, len(sections), 2):
        key = sections[i].strip()
        value = sections[i + 1].strip()
        review_dict[key] = value

    return review_dict


In [152]:
gold_standard = {item['paper_id']: parse_review1(item['response']) for item in gold_standard}
predictions = {item['paper_id']: parse_review1(item['response']) for item in predictions}


TypeError: string indices must be integers

In [215]:
with open('matched_papers_reviews_test.json', 'r') as f:
    papers_2025 = json.load(f)

In [216]:
def remove_citations(full_text):
    idx = full_text.find('\n\nREFERENCES')
    if idx == -1:
        idx = full_text.find('\n\nReferences')
    return full_text[:idx]

In [217]:
with open('instruction_finetuning_data.jsonl', 'r') as f:
    train_data = [json.loads(line) for line in f]
with open('instruction_finetuning_data_test.jsonl', 'r') as f:
    test_data = [json.loads(line) for line in f]

In [218]:
prompt_string = """
Categories (1–5):
- Soundness: The rigor of the methods for the stated problem
- Presentation: The clarity of writing and organization
- Contribution: The paper’s novelty or added value to the domain

Additionally:
- Rating (1–10): Overall recommendation for acceptance
- Confidence (1–5): How confident the reviewer is in their assessment

Please provide:
1) Soundness
2) Presentation
3) Contribution
4) Rating
5) Confidence
6) Explanation (strengths, weaknesses)
7) Questions
"""

In [219]:
json_txt = [{'paper_id': item['paper_id'], 
             'summary': item['prompt'].replace(prompt_string, ''), 
             'full text': remove_citations(papers_2025[item['paper_id']]['full_text']['value']), 
             'abstract': papers_2025[item['paper_id']]['abstract']['value'],
             'response': parse_review1(item['response'])} 
             for item in test_data]

In [220]:
with open('instruction_finetuning_test_data_reformatted.json', "w", encoding="utf-8") as json_file:
    json.dump(json_txt, json_file, indent=4)

In [81]:
def confusion_matrix_gen(num_pred, num_gold):
    num_pred = np.array(num_pred)
    num_gold = np.array(num_gold)

    soundness_conf = confusion_matrix(num_pred[:,0], num_gold[:,0], labels=[1,2,3,4,5])
    presentation_conf = confusion_matrix(num_pred[:,1], num_gold[:,1], labels=[1,2,3,4,5])
    contribution_conf = confusion_matrix(num_pred[:,2], num_gold[:,2], labels=[1,2,3,4,5])
    rating_conf = confusion_matrix(num_pred[:,3], num_gold[:,3], labels=[1,2,3,4,5,6,7,8,9,10])
    confidence_conf = confusion_matrix(num_pred[:,4], num_gold[:,4], labels=[1,2,3,4,5])

    return(soundness_conf, presentation_conf, contribution_conf, rating_conf, confidence_conf)

In [82]:
predictions = {k: predictions[k] for k in ('aAcOaJYbUg', 'dgR6i4TSng', 'UatDdAlr2x')}
predictions

{'aAcOaJYbUg': {'Soundness': 3,
  'Presentation': 3,
  'Contribution': 4,
  'Rating': 6,
  'Confidence': 3,
  'Strengths': '1. Clear structure and presentation: The paper is well-organized, with a logical structure and smooth flow that improves readability and comprehension.\n2. Insightful motivation for OOD benchmarking: This dataset addresses the critical issue of data leakage in current benchmarks. This approach highlights the limitations of evaluating current models trained on web-scale data.\n3. Interesting distortion design: The authors designed six distinct distortion types that minimize overlap with common real-world corruptions, reducing the likelihood of data leakage. This approach improves the dataset’s utility as a truly challenging OOD benchmark.',
  'Weaknesses': "1. Limited novel insights in model ranking comparisons: Figure 4 shows minimal divergence in the ranking of model performance between ImageNet-C and LAION-C. Similarly, Figure 3 demonstrates a strong linear corr

In [83]:
#def evaluate_review(predictions, gold_standard):
results = {}
numerical_scores = ["Soundness", "Presentation", "Contribution", "Rating", "Confidence"]
text_fields = ["Strengths", "Weaknesses", "Questions"]

num_pred = []
num_gold = []
for paper_id, response in predictions.items():
    gold = gold_standard[paper_id]
    results[paper_id] = {}

    num_pred = num_pred + [[response[category] for category in numerical_scores]]
    num_gold = num_gold + [[gold[category] for category in numerical_scores]]

    results[paper_id]["MAE"] = mean_absolute_error(num_gold, num_pred)
    results[paper_id]["MSE"] = mean_squared_error(num_gold, num_pred)

    text_results = {}
    #for text in text_fields:
    P, R, F1 = score([response[text_fields[0]], response[text_fields[1]], response[text_fields[2]]], 
                     [gold[text_fields[0]], gold[text_fields[1]], gold[text_fields[2]]], lang="en", rescale_with_baseline=True)
    text_results[text_fields[0] + "_BERTScore"] = F1[0]
    text_results[text_fields[1] + "_BERTScore"] = F1[1]
    text_results[text_fields[2] + "_BERTScore"] = F1[2]

    embeddings = model.encode([response[text_fields[0]], gold[text_fields[0]], response[text_fields[1]], gold[text_fields[1]], response[text_fields[1]], gold[text_fields[2]]])
    text_results[text_fields[0] + "_Embedding"] = (embeddings[0], embeddings[1])
    text_results[text_fields[1] + "_Embedding"] = (embeddings[2], embeddings[3])
    text_results[text_fields[2] + "_Embedding"] = (embeddings[4], embeddings[5])

    results[paper_id]['Text Comparisons'] = text_results
    
matrices = confusion_matrix_gen(num_pred, num_gold)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [350]:
[response[text_fields[0]], response[text_fields[1]], response[text_fields[2]]]

['* The paper is well-written, easy to follow, and provides sufficient analysis for its claims.\n* Discovers and Discusses two strategies which 1-layer transformers can use to perform the counting task (i.e., relation-based and inventory-based) and in doing so considers a range of possible hyperparameter selections that affect this strategic choice.\n* During its analysis further studies the role of BoS token showing that together with softmax in the counting task, can help models with smaller p and d sizes to still reach perfect accuracy.',
 '* Limited scope and applicability, specifically: (1) Only examines single-layer transformers, (2)Focuses on just one task (histogram counting)\n* No clear path to extending insights to more complicated settings more specifically multi-layer transformers\n* Lack of practical impact in the sense that it is hard to find a direct translation from this paper findings to real-world transformer design for real-world tasks',
 'Have you done any analysis 

In [84]:
model.encode([predictions['aAcOaJYbUg']['Strengths'], gold_standard['aAcOaJYbUg']['Strengths']]).shape

(2, 384)

In [8]:
score([predictions['aAcOaJYbUg']['Explanation']], [gold_standard['aAcOaJYbUg']['Explanation']], lang='en', device='mps')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(tensor([1.0000]), tensor([1.0000]), tensor([1.0000]))